In [1]:
import pandas as pd
import numpy as np
import sqlite3
from tqdm.notebook import tqdm

In [2]:
#db = sqlite3.connect('data/NPPES_Data_Dissemination.sqlite')
query = "SELECT * FROM hop"

with sqlite3.connect('data/NPPES_Data_Dissemination.sqlite') as db: 
    hop_sqlite = pd.read_sql(query, db)
    
hop_sqlite

,from_npi,to_npi,patient_count,transaction_count,average_day_wait,std_day_wait
0,1508085911,1730166125,58,67,23.925,43.923
1,1508167040,1730166125,51,51,28.196,52.876
2,1508863549,1730166125,340,391,18.302,42.422
3,1508867870,1730166125,50,79,12.658,26.402
4,1508011040,1730166224,132,145,8.579,28.053
...,...,...,...,...,...,...
34176933,1417037664,1497939599,36,106,19.330,42.407
34176934,1417194903,1497939599,22,70,16.629,30.598
34176935,1417406372,1497939599,21,65,20.123,37.750
34176936,1417064825,1497940605,75,79,10.418,34.744


In [3]:
#db = sqlite3.connect('data/NPPES_Data_Dissemination.sqlite')
query = "SELECT * FROM nppes"

with sqlite3.connect('data/NPPES_Data_Dissemination.sqlite') as db: 
    nppes_sqlite = pd.read_sql(query, db)
    
nppes_sqlite

,NPI,EntityTypeCode,ProviderOrganizationName(LegalBusinessName),ProviderLastName(LegalName),ProviderFirstName,ProviderMiddleName,ProviderNamePrefixText,ProviderNameSuffixText,ProviderCredentialText,ProviderFirstLineBusinessPracticeLocationAddress,ProviderSecondLineBusinessPracticeLocationAddress,ProviderBusinessPracticeLocationAddressCityName,ProviderBusinessPracticeLocationAddressStateName,ProviderBusinessPracticeLocationAddressPostalCode,HealthcareProviderTaxonomyCode,HealthcareProviderPrimaryTaxonomySwitch
0,1750384210,1.0,None,GILMER,CARISSIA,None,None,None,PHARMD,1410 S 4TH ST,None,NASHVILLE,AR,718523009,183500000X,Y
1,1750384210,1.0,None,GILMER,CARISSIA,None,None,None,PHARMD,1410 S 4TH ST,None,NASHVILLE,AR,718523009,None,None
2,1750384210,1.0,None,GILMER,CARISSIA,None,None,None,PHARMD,1410 S 4TH ST,None,NASHVILLE,AR,718523009,None,None
3,1750384210,1.0,None,GILMER,CARISSIA,None,None,None,PHARMD,1410 S 4TH ST,None,NASHVILLE,AR,718523009,None,None
4,1750384210,1.0,None,GILMER,CARISSIA,None,None,None,PHARMD,1410 S 4TH ST,None,NASHVILLE,AR,718523009,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9760360,1104850130,2.0,HEIKKI E KOSTAMAA MD PC,None,None,None,None,None,None,28 WHITE BRIDGE PIKE,STE. 208,NASHVILLE,TN,37205,None,None
9760361,1104850130,2.0,HEIKKI E KOSTAMAA MD PC,None,None,None,None,None,None,28 WHITE BRIDGE PIKE,STE. 208,NASHVILLE,TN,37205,None,None
9760362,1104850130,2.0,HEIKKI E KOSTAMAA MD PC,None,None,None,None,None,None,28 WHITE BRIDGE PIKE,STE. 208,NASHVILLE,TN,37205,None,None
9760363,1104850130,2.0,HEIKKI E KOSTAMAA MD PC,None,None,None,None,None,None,28 WHITE BRIDGE PIKE,STE. 208,NASHVILLE,TN,37205,None,None


First, build a profile of providers referring patients to the major hospitals in Nashville. Are certain specialties more likely to refer to a particular hospital over the others?

In [4]:
db.close

<function Connection.close>

In [5]:
db = sqlite3.connect('data/NPPES_Data_Dissemination.sqlite')

providers_query = """

SELECT NPI, [ProviderOrganizationName(LegalBusinessName)], 
ProviderBusinessPracticeLocationAddressPostalCode, HealthcareProviderTaxonomyCode,
HealthcareProviderPrimaryTaxonomySwitch 

FROM [nppes] WHERE EntityTypeCode = 2 AND ProviderBusinessPracticeLocationAddressCityName = 'NASHVILLE' 
AND ProviderBusinessPracticeLocationAddressStateName = 'TN' AND HealthcareProviderTaxonomyCode IS NOT NULL
AND HealthcareProviderPrimaryTaxonomySwitch = 'Y'
"""
    
provider = pd.read_sql(providers_query, db)
    
provider

,NPI,ProviderOrganizationName(LegalBusinessName),ProviderBusinessPracticeLocationAddressPostalCode,HealthcareProviderTaxonomyCode,HealthcareProviderPrimaryTaxonomySwitch
0,1538270673,METROPOLITAN GOVERNMENT OF NASHVILLE AND DAVID...,372031503,251K00000X,Y
1,1891834198,"RADNOR MEDICAL GROUP, INC.",372115869,208000000X,Y
2,1700900040,JEFFREY D HORN MD PLLC,372032000,207W00000X,Y
3,1346440781,STERLING PRIMARY CARE 2LLC,372031582,261Q00000X,Y
4,1548450513,"MARY A. MCELANEY, M.D.",372114854,207RP1001X,Y
...,...,...,...,...,...
2979,1720785884,"EMERGENCY MEDICINE SERVICES OF TN, PC",372031538,207P00000X,Y
2980,1366193013,"COMFORT MEDICAL, LLC",372172009,332B00000X,Y
2981,1982301065,"HOSPITAL MEDICINE SERVICES OF TN, LLC",372031538,208M00000X,Y
2982,1710638523,"VORI MEDICAL NY, PLLC",372043622,208100000X,Y


In [6]:
db.close

<function Connection.close>

In [7]:
taxonomy_nucc = pd.read_csv('data/nucc_taxonomy_230.csv', encoding= 'unicode_escape')

##taxonomy_nucc1 = taxonomy_nucc[taxonomy_nucc['Grouping'] == 'Hospital Units']
taxonomy_nucc

,Code,Grouping,Classification,Specialization,Definition,Notes,Display Name,Section
0,193200000X,Group,Multi-Specialty,NaN,A business group of one or more individual pra...,[7/1/2003: new],Multi-Specialty Group,Individual
1,193400000X,Group,Single Specialty,NaN,A business group of one or more individual pra...,[7/1/2003: new],Single Specialty Group,Individual
2,207K00000X,Allopathic & Osteopathic Physicians,Allergy & Immunology,NaN,An allergist-immunologist is trained in evalua...,"Source: American Board of Medical Specialties,...",Allergy & Immunology Physician,Individual
3,207KA0200X,Allopathic & Osteopathic Physicians,Allergy & Immunology,Allergy,"A physician who specializes in the diagnosis, ...",Source: National Uniform Claim Committee,Allergy Physician,Individual
4,207KI0005X,Allopathic & Osteopathic Physicians,Allergy & Immunology,Clinical & Laboratory Immunology,An allergy and immunology physician who specia...,"Source: National Uniform Claim Committee, 2022...",Clinical & Laboratory Immunology (Allergy & Im...,Individual
...,...,...,...,...,...,...,...,...
868,343800000X,Transportation Services,Secured Medical Transport (VAN),NaN,A public or privately owned transportation ser...,NaN,Secured Medical Transport (VAN),Non-Individual
869,344600000X,Transportation Services,Taxi,NaN,A land commercial vehicle used for the transpo...,NaN,Taxi,Non-Individual
870,347D00000X,Transportation Services,Train,NaN,An organization or business licensed to provid...,NaN,Train,Non-Individual
871,347E00000X,Transportation Services,Transportation Broker,NaN,An organization that provides transportation f...,Source: Section 6083 of the Deficit Reduction ...,Transportation Broker,Non-Individual


In [8]:
provider = pd.DataFrame(provider)
provider = provider.merge(taxonomy_nucc, left_on = "HealthcareProviderTaxonomyCode", right_on = "Code")
provider_hospitals = provider[provider['Grouping'] == 'Hospitals'] 
provider_hospitals

,NPI,ProviderOrganizationName(LegalBusinessName),ProviderBusinessPracticeLocationAddressPostalCode,HealthcareProviderTaxonomyCode,HealthcareProviderPrimaryTaxonomySwitch,Code,Grouping,Classification,Specialization,Definition,Notes,Display Name,Section
1024,1609056399,VANDERBILT CHILDREN'S,372320001,282NC2000X,Y,282NC2000X,Hospitals,General Acute Care Hospital,Children,Definition to come...,NaN,Children's Hospital,Non-Individual
1025,1326224015,VANDERBILT CHILDRENS HOSPITAL,372320005,282NC2000X,Y,282NC2000X,Hospitals,General Acute Care Hospital,Children,Definition to come...,NaN,Children's Hospital,Non-Individual
1026,1164609319,VANDERBILT MEDICAL CENTER,372320034,282NC2000X,Y,282NC2000X,Hospitals,General Acute Care Hospital,Children,Definition to come...,NaN,Children's Hospital,Non-Individual
1027,1255577466,MONROE CARELL JR VANDERBILT CHILDREN'S HOSPITAL,372320005,282NC2000X,Y,282NC2000X,Hospitals,General Acute Care Hospital,Children,Definition to come...,NaN,Children's Hospital,Non-Individual
1028,1003211145,MONROE CARELL JR. CHILDREN'S HOSPITAL AT VANDE...,372320005,282NC2000X,Y,282NC2000X,Hospitals,General Acute Care Hospital,Children,Definition to come...,NaN,Children's Hospital,Non-Individual
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2933,1528033404,TENNESSEE VALLEY HEALTH CARE9(TVHS),372122637,284300000X,Y,284300000X,Hospitals,Special Hospital,NaN,A designation by the AHA of a hospital whose p...,"Source: AHA Guide, Registration section, p. A5.",Special Hospital,Non-Individual
2934,1336119627,BAPTIST WOMEN'S HEALTH CENTER LLC,372032065,284300000X,Y,284300000X,Hospitals,Special Hospital,NaN,A designation by the AHA of a hospital whose p...,"Source: AHA Guide, Registration section, p. A5.",Special Hospital,Non-Individual
2936,1568477990,DEPARTMENT OF VETERANS AFFAIRS NASHVILLE,372122637,286500000X,Y,286500000X,Hospitals,Military Hospital,NaN,A health care facility operated by the Departm...,NaN,Military Hospital,Non-Individual
2960,1215937982,SELECT SPECIALTY HOSPITAL - NASHVILLE LLC,372360001,282E00000X,Y,282E00000X,Hospitals,Long Term Care Hospital,NaN,Long-term care hospitals (LTCHs) furnish exten...,Source: American Hospital Association [7/1/20...,Long Term Care Hospital,Non-Individual


Merging data to be able to get 3. question done. 

In [9]:
provider

#provider[provider['ProviderOrganizationName(LegalBusinessName)'] == 'VANDERBILT MEDICAL CENTER'] 

,NPI,ProviderOrganizationName(LegalBusinessName),ProviderBusinessPracticeLocationAddressPostalCode,HealthcareProviderTaxonomyCode,HealthcareProviderPrimaryTaxonomySwitch,Code,Grouping,Classification,Specialization,Definition,Notes,Display Name,Section
0,1538270673,METROPOLITAN GOVERNMENT OF NASHVILLE AND DAVID...,372031503,251K00000X,Y,251K00000X,Agencies,Public Health or Welfare,NaN,Definition to come...,NaN,Public Health or Welfare Agency,Non-Individual
1,1356504153,TENNESSE DEPARTMENT OF HEALTH,372470001,251K00000X,Y,251K00000X,Agencies,Public Health or Welfare,NaN,Definition to come...,NaN,Public Health or Welfare Agency,Non-Individual
2,1225277783,METROPOLITAN NASHVILLE PUBLIC HEALTH DEPARTMENT,372031503,251K00000X,Y,251K00000X,Agencies,Public Health or Welfare,NaN,Definition to come...,NaN,Public Health or Welfare Agency,Non-Individual
3,1265594022,"TECHNOVISION, LLC.",372032941,251K00000X,Y,251K00000X,Agencies,Public Health or Welfare,NaN,Definition to come...,NaN,Public Health or Welfare Agency,Non-Individual
4,1396058269,MEHARRY MEDICAL COLLEGE,372082918,251K00000X,Y,251K00000X,Agencies,Public Health or Welfare,NaN,Definition to come...,NaN,Public Health or Welfare Agency,Non-Individual
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2979,1629598453,"FEMALE PHYSICIANS OF TENNESSEE, P.C.",372031515,2088F0040X,Y,2088F0040X,Allopathic & Osteopathic Physicians,Urology,Female Pelvic Medicine and Reconstructive Surgery,A subspecialist in Female Pelvic Medicine and ...,"Source: American Board of Medical Specialties,...",Female Pelvic Medicine and Reconstructive Surg...,Individual
2980,1477186336,NATIONAL ADDICTION SPECIALISTS LLC,372142403,207RA0401X,Y,207RA0401X,Allopathic & Osteopathic Physicians,Internal Medicine,Addiction Medicine,An internist doctor of osteopathy that special...,Source: American Osteopathic Board of Internal...,Addiction Medicine (Internal Medicine) Physician,Individual
2981,1528778263,EMPOWER MUSIC THERAPY LLC,372281906,225A00000X,Y,225A00000X,"Respiratory, Developmental, Rehabilitative and...",Music Therapist,NaN,Music therapists use music interventions to as...,Source: American Music Therapy Association,Music Therapist,Individual
2982,1053069559,MIDDLE TENNESSEE IMAGING LLC,372032220,261QR0206X,Y,261QR0206X,Ambulatory Health Care Facilities,Clinic/Center,"Radiology, Mammography",Definition to come...,NaN,Mammography Clinic/Center,Non-Individual



- Determine which professionals Vanderbilt Hospital should reach out to in the Nashville area to expand their own patient volume.
First, research which professionals are sending significant numbers of patients only to competitor hospitals (such as TriStar Centennial Medical Center).

In [11]:
db = sqlite3.connect('data/NPPES_Data_Dissemination.sqlite')

competitors = """
SELECT h.from_npi, h.to_npi, h.patient_count, h.transaction_count, ProviderBusinessPracticeLocationAddressCityName AS city_name, [ProviderOrganizationName(LegalBusinessName)] AS Business_name, 
ProviderFirstName AS First_Name, [ProviderLastName(LegalName)] AS Last_Name
        
FROM nppes AS n
INNER JOIN hop AS h
ON n.NPI = h.to_npi



"""
with sqlite3.connect('data/NPPES_Data_Dissemination.sqlite') as db: 
    competitors = pd.read_sql(competitors, db)
    

In [13]:
competitors

,from_npi,to_npi,patient_count,transaction_count,city_name,Business_name,First_Name,Last_Name
0,1295726032,1922001957,83,89,NASHVILLE,None,RICHARD,PRESLEY
1,1578584199,1922001957,129,149,NASHVILLE,None,RICHARD,PRESLEY
2,1295726032,1922001957,83,89,NASHVILLE,None,RICHARD,PRESLEY
3,1578584199,1922001957,129,149,NASHVILLE,None,RICHARD,PRESLEY
4,1295726032,1922001957,83,89,NASHVILLE,None,RICHARD,PRESLEY
...,...,...,...,...,...,...,...,...
53854135,1811955917,1104850130,52,90,NASHVILLE,HEIKKI E KOSTAMAA MD PC,None,None
53854136,1861479545,1104850130,195,287,NASHVILLE,HEIKKI E KOSTAMAA MD PC,None,None
53854137,1871529982,1104850130,80,81,NASHVILLE,HEIKKI E KOSTAMAA MD PC,None,None
53854138,1922039346,1104850130,912,2042,NASHVILLE,HEIKKI E KOSTAMAA MD PC,None,None


In [12]:
db = sqlite3.connect('data/NPPES_Data_Dissemination.sqlite')

hospitals = """
SELECT NPI, h.from_npi, h.to_npi, 
h.patient_count, h.transaction_count, 
Specialization,
p.[ProviderOrganizationName(LegalBusinessName)] AS Business_name, 
Grouping
        
FROM provider AS p
LEFT JOIN hop AS h
ON p.NPI = h.to_npi
WHERE Specialization IS NOT NULL
ORDER BY h.patient_count DESC;

"""
with sqlite3.connect('data/NPPES_Data_Dissemination.sqlite') as db: 
    hospitals = pd.read_sql(hospitals, db)

In [16]:
hospitals['Business_name'].unique()

array(['ASSOCIATED PATHOLOGISTS, LLC', 'RADIOLOGY ALLIANCE PC',
       'TENNESSEE ONCOLOGY PLLC', 'MIDDLE TENNESSEE IMAGING LLC',
       'PATHOLOGY ASSOCIATES OF ST. THOMAS', 'NEPHROLOGY ASSOCIATES, PC',
       'CARDIOVASCULAR ANESTHESIOLOGISTS PC', 'ACLA PC',
       'ONE HUNDRED OAKS IMAGING,LLC',
       'WELLNESS SOLUTIONS GERIATRICS, PLLC',
       'VANDERBILT IMAGING SERVICES, LLC',
       'MEDICONE MEDICAL RESPONSE OF MIDDLE TENNESSEE INC',
       'MIDDLE TENNESSEE PULMONARY ASSOCIA', 'TENNESSEE RETINA PC',
       'FIRST CALL AMBULANCE SERVICE, LLC', 'ACCURATE HEALTHCARE, INC.',
       'CCF TENNESSEE MEDICAL SERVICES PC', 'TENNESSEE ONCOLOGY,PLLC',
       'UROLOGY SURGERY CENTER LP', 'MEDICAL NECESSITIES & SERVICES LLC',
       'AEPC ANESTHESIA LLC', 'SKYLINE NEUROSCIENCE ASSOCIATES, LLC',
       'RETAIL HEALTH CLINICS, LLC', 'SAINT THOMAS CAMPUS SURGICARE LP',
       'DIGESTIVE DISEASE ENDOSCOPY CENTER, INC.',
       'EYE SURGERY CENTER OF NASHVILLE LLC', 'BAPTIST SURGERY CENTER L

In [15]:
hospitals = pd.DataFrame(hospitals)
hospitals

,NPI,from_npi,to_npi,patient_count,transaction_count,Specialization,Business_name,Grouping
0,1003863580,1.124076e+09,1.003864e+09,192118.0,316456.0,Anatomic Pathology & Clinical Pathology,"ASSOCIATED PATHOLOGISTS, LLC",Allopathic & Osteopathic Physicians
1,1003863580,1.093753e+09,1.003864e+09,127792.0,234741.0,Anatomic Pathology & Clinical Pathology,"ASSOCIATED PATHOLOGISTS, LLC",Allopathic & Osteopathic Physicians
2,1003863580,1.235187e+09,1.003864e+09,118714.0,212053.0,Anatomic Pathology & Clinical Pathology,"ASSOCIATED PATHOLOGISTS, LLC",Allopathic & Osteopathic Physicians
3,1003863580,1.609849e+09,1.003864e+09,29321.0,30593.0,Anatomic Pathology & Clinical Pathology,"ASSOCIATED PATHOLOGISTS, LLC",Allopathic & Osteopathic Physicians
4,1861478489,1.245393e+09,1.861478e+09,19602.0,36881.0,Diagnostic Radiology,RADIOLOGY ALLIANCE PC,Allopathic & Osteopathic Physicians
...,...,...,...,...,...,...,...,...
26533,1679007017,NaN,NaN,NaN,NaN,Rural Health,"AMERICA CARES TRUST, INC (ACT)",Ambulatory Health Care Facilities
26534,1467961896,NaN,NaN,NaN,NaN,Child Abuse Pediatrics,OUR KIDS,Allopathic & Osteopathic Physicians
26535,1003534165,NaN,NaN,NaN,NaN,Exercise & Sports,MINDFLOW LLC,Behavioral Health & Social Service Providers
26536,1477186336,NaN,NaN,NaN,NaN,Addiction Medicine,NATIONAL ADDICTION SPECIALISTS LLC,Allopathic & Osteopathic Physicians


In [21]:
df  = hospitals.loc[hospitals['Business_name'].str.startswith('VANDERBILT', na=False)]
df

,NPI,from_npi,to_npi,patient_count,transaction_count,Specialization,Business_name,Grouping
98,1144241985,1.104203e+09,1.144242e+09,3839.0,4702.0,Diagnostic Radiology,"VANDERBILT IMAGING SERVICES, LLC",Allopathic & Osteopathic Physicians
104,1144241985,1.396882e+09,1.144242e+09,3706.0,4542.0,Diagnostic Radiology,"VANDERBILT IMAGING SERVICES, LLC",Allopathic & Osteopathic Physicians
506,1285091330,1.104203e+09,1.285091e+09,1199.0,5498.0,Community/Retail Pharmacy,VANDERBILT UNIVERSITY MEDICAL CENTER,Suppliers
510,1285091330,1.396882e+09,1.285091e+09,1194.0,5359.0,Community/Retail Pharmacy,VANDERBILT UNIVERSITY MEDICAL CENTER,Suppliers
837,1144241985,1.295822e+09,1.144242e+09,814.0,891.0,Diagnostic Radiology,"VANDERBILT IMAGING SERVICES, LLC",Allopathic & Osteopathic Physicians
...,...,...,...,...,...,...,...,...
26324,1679971485,NaN,NaN,NaN,NaN,Specialty Pharmacy,VANDERBILT UNIVERSITY MEDICAL CENTER,Suppliers
26325,1962922435,NaN,NaN,NaN,NaN,Specialty Pharmacy,VANDERBILT HEALTH PHARMACY GROUP LLC,Suppliers
26410,1427394634,NaN,NaN,NaN,NaN,Critical Care Medicine,VANDERBILT UNIVERSITY MEDICAL CENTER,Physician Assistants & Advanced Practice Nursi...
26456,1700800877,NaN,NaN,NaN,NaN,Institutional Pharmacy,VANDERBILT UNIVERSITY MEDICAL CENTER,Suppliers


In [22]:
newdf = hospitals[hospitals['Business_name'] != 'VANDERBILT UNIVERSITY MEDICAL CENTER' ]

In [24]:
newdf

,NPI,from_npi,to_npi,patient_count,transaction_count,Specialization,Business_name,Grouping
0,1003863580,1.124076e+09,1.003864e+09,192118.0,316456.0,Anatomic Pathology & Clinical Pathology,"ASSOCIATED PATHOLOGISTS, LLC",Allopathic & Osteopathic Physicians
1,1003863580,1.093753e+09,1.003864e+09,127792.0,234741.0,Anatomic Pathology & Clinical Pathology,"ASSOCIATED PATHOLOGISTS, LLC",Allopathic & Osteopathic Physicians
2,1003863580,1.235187e+09,1.003864e+09,118714.0,212053.0,Anatomic Pathology & Clinical Pathology,"ASSOCIATED PATHOLOGISTS, LLC",Allopathic & Osteopathic Physicians
3,1003863580,1.609849e+09,1.003864e+09,29321.0,30593.0,Anatomic Pathology & Clinical Pathology,"ASSOCIATED PATHOLOGISTS, LLC",Allopathic & Osteopathic Physicians
4,1861478489,1.245393e+09,1.861478e+09,19602.0,36881.0,Diagnostic Radiology,RADIOLOGY ALLIANCE PC,Allopathic & Osteopathic Physicians
...,...,...,...,...,...,...,...,...
26533,1679007017,NaN,NaN,NaN,NaN,Rural Health,"AMERICA CARES TRUST, INC (ACT)",Ambulatory Health Care Facilities
26534,1467961896,NaN,NaN,NaN,NaN,Child Abuse Pediatrics,OUR KIDS,Allopathic & Osteopathic Physicians
26535,1003534165,NaN,NaN,NaN,NaN,Exercise & Sports,MINDFLOW LLC,Behavioral Health & Social Service Providers
26536,1477186336,NaN,NaN,NaN,NaN,Addiction Medicine,NATIONAL ADDICTION SPECIALISTS LLC,Allopathic & Osteopathic Physicians
